In [1]:
import os
from maze_env import Maze
from utility import Prediction
import time

In [2]:
# def onKeyPress(event):
#     is_terminate = False
#     if (event.char == "w"):
#         is_terminate, reward = env.take_action(0, show_animate=True)
#     elif (event.char == "d"):
#         is_terminate, reward = env.take_action(1, show_animate=True)
#     elif (event.char == "s"):
#         is_terminate, reward = env.take_action(2, show_animate=True)
#     elif (event.char == "a"):
#         is_terminate, reward = env.take_action(3, show_animate=True)
#     elif (event.char == "q"):
#         env.destroy()
#     if (is_terminate):
#         env.reset()

In [3]:
# if __name__ == "__main__":
env = Maze()
prd = Prediction(env)
print('{0}x{1}, act_space = {2}'.format(env.MAZE_Limit[0],env.MAZE_Limit[1], len(env.action_space)))
# env.bind('<KeyPress>', onKeyPress)
# env.mainloop()

4x4, act_space = 4


# DP Prediction

In [4]:
# prd.DP.iteration(update = 10)

# model-free MC

In [5]:
prd.MFMC.iteration(n_episode = 3000,  mode = "first_visit")
prd.MFMC.iteration(n_episode = 3000,  mode = "every_visit")

The mode is first_visit
N
 [[   0. 1809. 2018. 1451.]
 [1198. 2081. 3000. 1916.]
 [ 855. 1068.    0. 1246.]
 [ 602.  737.  457.    0.]]
V
 [[  0.         -14.17910448 -19.87809713 -21.77877326]
 [-14.03005008 -18.59346468 -20.676      -19.24843424]
 [-20.44093567 -20.59082397   0.         -11.18539326]
 [-21.99335548 -19.28629579 -11.81400438   0.        ]]
--------------------------

The mode is every_visit
N
 [[    0.  4207.  7249.  7217.]
 [ 2944.  5447. 10688.  7057.]
 [ 3327.  3771.     0.  3335.]
 [ 2997.  2685.  1341.     0.]]
V
 [[  0.         -13.56928928 -19.63815699 -20.93501455]
 [-14.40081522 -18.39324399 -20.43151198 -18.91143545]
 [-19.90862639 -20.1516839    0.         -11.76101949]
 [-21.19252586 -18.63128492 -10.43400447   0.        ]]
--------------------------

